In [ ]:
# Methodology used:
# Combining correlated features using PCA
# Converting categories into numerics and scaling all features
# Using random forest with n=500
# Using cross validation with K=10 to check the score

In [17]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import accuracy_score
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline
import plotly.plotly as py
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition
from sklearn import datasets
import xgboost as xgb
from sklearn.neighbors import NearestNeighbors

In [69]:
# Importing the data

train=pd.read_csv("water_train.csv")
test=pd.read_csv("test_set.csv")
label=pd.read_csv("water_train_results.csv")

print("Train size:",train.shape)
print("Test size:",test.shape)
print("Label size:",label.shape)

Train size: (59400, 40)
Test size: (14850, 40)
Label size: (59400, 2)


In [61]:
pd.set_option('display.max_columns', 500)
train.head(10)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,14/03/2011,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,06/03/2013,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,25/02/2013,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,28/01/2013,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,13/07/2011,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
5,9944,20.0,13/03/2011,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,Pangani,Moa/Mwereme,Tanga,4,8,Mkinga,Moa,1,True,GeoData Consultants Ltd,VWC,Zingibali,True,2009,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe
6,19816,0.0,01/10/2012,Dwsp,0,DWSP,33.362410,-3.766365,Kwa Ngomho,0,Internal,Ishinabulandi,Shinyanga,17,3,Shinyanga Rural,Samuye,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,swn 80,swn 80,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
7,54551,0.0,09/10/2012,Rwssp,0,DWE,32.620617,-4.226198,Tushirikiane,0,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
8,53934,0.0,03/11/2012,Wateraid,0,Water Aid,32.711100,-5.146712,Kwa Ramadhan Musa,0,Lake Tanganyika,Imalauduki,Tabora,14,6,Tabora Urban,Itetemia,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump
9,46144,0.0,03/08/2011,Isingiro Ho,0,Artisan,30.626991,-1.257051,Kwapeto,0,Lake Victoria,Mkonomre,Kagera,18,1,Karagwe,Kaisho,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


# Converting Categories to Numeric

In [66]:
#Converting the categorical variables into strings
for col in train.select_dtypes(['object']):
    train[col] = train[col].astype(str)
    
for col in test.select_dtypes(['object']):
    test[col] = test[col].astype(str)
    
for col in train.select_dtypes(['object']):
    train[col] = train[col].str.lower()
    
#Replacing categorical variables with corresponding numerical variables
for col in train.select_dtypes(['object']):
    train[col] = preprocessing.LabelEncoder().fit_transform(train[col])
    
for col in test.select_dtypes(['object']):
    test[col] = preprocessing.LabelEncoder().fit_transform(test[col])
    
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,184,1370,1390,1411,34.938093,-9.856322,32436,0,1,11807,3,11,5,51,1426,109,2,0,8,2199,0,1999,3,1,0,7,4,2,0,6,2,1,1,8,6,0,1,1
1,8776,0.0,95,469,1399,531,34.698766,-2.147466,37195,0,4,15839,9,20,2,103,1576,280,1,0,3,1868,2,2010,3,1,0,11,4,0,2,6,2,2,2,5,3,1,1,1
2,34310,25.0,304,825,686,1906,37.460664,-3.821329,14572,0,5,9074,8,21,4,108,1624,250,2,0,8,2082,2,2009,3,1,0,7,4,4,5,6,2,1,1,0,1,1,2,1
3,67743,0.0,329,1742,263,1754,38.486161,-11.155298,37285,0,7,8982,12,90,63,87,1571,58,2,0,8,1868,2,1986,14,10,5,7,4,0,2,6,2,0,0,3,0,0,2,1
4,19728,0.0,175,20,0,104,31.130847,-1.825359,35529,0,4,7698,4,18,1,26,1687,0,2,0,1,1868,2,0,3,1,0,1,1,0,2,6,2,3,3,5,3,1,1,1


In [71]:
#Converting the categorical variables into strings
temp = pd.merge(temp, label, how='inner', on='id')
for col in temp.select_dtypes(['object']):
    temp[col] = temp[col].astype(str)
    
for col in temp.select_dtypes(['object']):
    temp[col] = temp[col].str.lower()
    
#Replacing categorical variables with corresponding numerical variables
for col in temp.select_dtypes(['object']):
    temp[col] = preprocessing.LabelEncoder().fit_transform(temp[col])


In [35]:
#Removing id column
del train['id']
del test['id']

# Scaling All Features

In [36]:
scaler = preprocessing.MinMaxScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train), columns=train.columns)

test_scaled = pd.DataFrame(scaler.fit_transform(test), columns=test.columns)

train_scaled.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,0.017143,0.518310,0.722193,0.517483,0.729199,0.865979,0.153923,0.867319,0.0,0.125,0.612174,0.15,0.102041,0.0625,0.411290,0.681970,0.003574,1.0,0.0,0.666667,0.853649,0.0,0.993045,0.176471,0.083333,0.000000,0.636364,1.00,0.333333,0.000000,0.857143,0.4,0.25,0.25,0.888889,1.000000,0.0,0.166667,0.2
1,0.000000,0.267606,0.247232,0.520629,0.274419,0.860047,0.815659,0.994572,0.0,0.500,0.821227,0.45,0.193878,0.0250,0.830645,0.753706,0.009180,0.5,0.0,0.250000,0.725155,1.0,0.998510,0.176471,0.083333,0.000000,1.000000,1.00,0.000000,0.333333,0.857143,0.4,0.50,0.50,0.555556,0.500000,0.5,0.166667,0.2
2,0.000071,0.856338,0.434897,0.271329,0.985013,0.928504,0.671973,0.389647,0.0,0.625,0.470472,0.40,0.204082,0.0500,0.870968,0.776662,0.008197,1.0,0.0,0.666667,0.808230,1.0,0.998013,0.176471,0.083333,0.000000,0.636364,1.00,0.666667,0.833333,0.857143,0.4,0.25,0.25,0.000000,0.166667,0.5,0.333333,0.2
3,0.000000,0.926761,0.918292,0.123427,0.906460,0.953922,0.042418,0.996978,0.0,0.875,0.465702,0.60,0.908163,0.7875,0.701613,0.751315,0.001902,1.0,0.0,0.666667,0.725155,1.0,0.986587,0.823529,0.833333,0.833333,0.636364,1.00,0.000000,0.333333,0.857143,0.4,0.00,0.00,0.333333,0.000000,0.0,0.333333,0.2
4,0.000000,0.492958,0.010543,0.031469,0.053747,0.771612,0.843309,0.950024,0.0,0.500,0.399129,0.20,0.173469,0.0125,0.209677,0.806791,0.000000,1.0,0.0,0.083333,0.725155,1.0,0.000000,0.176471,0.083333,0.000000,0.090909,0.25,0.000000,0.333333,0.857143,0.4,0.75,0.75,0.555556,0.500000,0.5,0.166667,0.2


In [37]:
train3 = train_scaled
test3 = test_scaled

In [38]:
# PCA for funder and installer
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(train_scaled[['funder', 'installer']])
train3['funder and installer pc'] = pd.DataFrame(data = principalComponents)

#Removing funder and installer columns
del train3['funder']
del train3['installer']

train3.head(10)


pca = PCA(n_components=1)
principalComponents = pca.fit_transform(test_scaled[['funder', 'installer']])
test3['funder and installer pc'] = pd.DataFrame(data = principalComponents)

#Removing funder and installer columns
del test3['funder']
del test3['installer']

test3.head(10)

,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,funder and installer pc
0,0.00000,0.772727,0.724418,0.875159,0.648955,0.060522,0.0,0.000,0.463816,0.40,0.204082,0.0375,0.508065,0.008682,0.027988,0.5,0.0,0.181818,0.990497,0.5,0.999503,0.5625,0.500000,0.500000,0.272727,0.5,0.000000,0.333333,0.857143,0.4,0.75,0.75,0.555556,0.500000,0.5,1.000000,1.0,-0.358319
1,0.00000,0.772727,0.573747,0.909031,0.713849,0.164222,0.0,0.625,0.321805,0.00,0.010204,0.0250,0.000000,0.331971,0.026157,0.5,0.0,0.545455,0.899944,0.5,0.993542,0.1875,0.083333,0.000000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.50,0.50,0.888889,1.000000,0.0,0.166667,0.2,-0.296414
2,0.00000,0.763636,0.573042,0.862191,0.567270,0.892426,0.0,0.000,0.639346,0.90,0.122449,0.0250,0.879032,0.856486,0.043596,0.5,0.0,0.545455,0.812186,1.0,0.998510,0.5625,0.500000,0.500000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.50,0.50,0.555556,0.500000,0.5,1.000000,1.0,0.777642
3,0.00000,0.733333,0.114326,0.943783,0.185559,0.516192,0.0,0.875,0.331043,0.35,0.806122,0.5375,0.395161,0.608784,0.021798,1.0,0.0,0.545455,0.990497,0.5,0.987084,0.5625,0.500000,0.500000,0.636364,1.0,1.000000,1.000000,0.857143,0.4,0.00,0.00,0.777778,0.833333,0.0,1.000000,1.0,-0.293258
4,0.00250,0.927273,0.464714,0.868099,0.053109,0.526432,0.0,0.875,0.419756,0.80,0.091837,0.0375,0.491935,0.549030,0.005231,1.0,0.0,0.818182,0.042482,0.5,0.993542,0.1875,0.083333,0.000000,0.818182,1.0,0.500000,0.166667,0.857143,0.4,0.25,0.25,0.888889,1.000000,0.0,0.166667,0.2,-0.520495
5,0.00000,0.857576,0.614679,0.909740,0.714437,0.728850,0.0,0.625,0.511548,0.00,0.010204,0.0250,0.000000,0.176711,0.017438,0.5,0.0,0.545455,0.912241,0.5,0.988574,0.1875,0.083333,0.000000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.25,0.25,0.888889,1.000000,0.0,0.166667,0.2,-0.296414
6,0.00000,0.103030,0.214185,0.902617,0.347890,1.000000,0.0,0.750,0.287457,0.15,0.102041,0.0875,0.274194,0.475485,0.052315,0.5,0.0,0.545455,0.990497,0.5,0.997019,0.2500,0.166667,0.166667,0.636364,1.0,0.000000,0.333333,0.571429,0.8,0.25,0.25,0.333333,0.000000,0.0,0.666667,0.6,-0.247027
7,0.00000,0.742424,0.102682,0.982205,0.058005,0.432697,0.0,0.875,0.162501,0.60,0.081633,0.0500,0.935484,0.535240,0.000087,0.5,0.0,0.818182,0.054779,0.5,0.984600,0.8125,0.833333,0.833333,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.00,0.00,0.333333,0.000000,0.0,0.333333,0.2,-0.285863
8,0.00015,0.736364,0.226182,0.973662,0.064075,0.503275,0.0,0.875,0.023807,0.60,0.908163,0.4125,0.709677,0.621553,0.003488,0.5,0.0,0.545455,0.428172,0.0,0.992052,0.1875,0.083333,0.000000,0.636364,1.0,0.666667,0.833333,0.857143,0.4,0.50,0.50,0.888889,1.000000,0.0,0.166667,0.2,0.348583
9,0.00000,0.900000,0.402258,0.919928,0.718818,0.279638,0.0,0.625,0.096411,0.30,0.020408,0.0875,0.854839,0.907559,0.000087,0.5,0.0,0.818182,0.011738,0.5,0.995032,0.1875,0.083333,0.000000,0.818182,1.0,0.500000,0.166667,0.857143,0.4,0.25,0.25,0.888889,1.000000,0.0,0.166667,0.2,-0.009892


In [39]:
# PCA for waterpoint type and waterpoint type group
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(train_scaled[['waterpoint_type', 'waterpoint_type_group']])
train3['waterpoint_type pc'] = pd.DataFrame(data = principalComponents)

#Removing waterpoint columns
del train3['waterpoint_type']
del train3['waterpoint_type_group']

train3.head(10)

pca = PCA(n_components=1)
principalComponents = pca.fit_transform(test_scaled[['waterpoint_type', 'waterpoint_type_group']])
test3['waterpoint_type pc'] = pd.DataFrame(data = principalComponents)

#Removing waterpoint columns
del test3['waterpoint_type']
del test3['waterpoint_type_group']

test3.head(10)

,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,funder and installer pc,waterpoint_type pc
0,0.00000,0.772727,0.724418,0.875159,0.648955,0.060522,0.0,0.000,0.463816,0.40,0.204082,0.0375,0.508065,0.008682,0.027988,0.5,0.0,0.181818,0.990497,0.5,0.999503,0.5625,0.500000,0.500000,0.272727,0.5,0.000000,0.333333,0.857143,0.4,0.75,0.75,0.555556,0.500000,0.5,-0.358319,0.816794
1,0.00000,0.772727,0.573747,0.909031,0.713849,0.164222,0.0,0.625,0.321805,0.00,0.010204,0.0250,0.000000,0.331971,0.026157,0.5,0.0,0.545455,0.899944,0.5,0.993542,0.1875,0.083333,0.000000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.50,0.50,0.888889,1.000000,0.0,-0.296414,-0.338225
2,0.00000,0.763636,0.573042,0.862191,0.567270,0.892426,0.0,0.000,0.639346,0.90,0.122449,0.0250,0.879032,0.856486,0.043596,0.5,0.0,0.545455,0.812186,1.0,0.998510,0.5625,0.500000,0.500000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.50,0.50,0.555556,0.500000,0.5,0.777642,0.816794
3,0.00000,0.733333,0.114326,0.943783,0.185559,0.516192,0.0,0.875,0.331043,0.35,0.806122,0.5375,0.395161,0.608784,0.021798,1.0,0.0,0.545455,0.990497,0.5,0.987084,0.5625,0.500000,0.500000,0.636364,1.0,1.000000,1.000000,0.857143,0.4,0.00,0.00,0.777778,0.833333,0.0,-0.293258,0.816794
4,0.00250,0.927273,0.464714,0.868099,0.053109,0.526432,0.0,0.875,0.419756,0.80,0.091837,0.0375,0.491935,0.549030,0.005231,1.0,0.0,0.818182,0.042482,0.5,0.993542,0.1875,0.083333,0.000000,0.818182,1.0,0.500000,0.166667,0.857143,0.4,0.25,0.25,0.888889,1.000000,0.0,-0.520495,-0.338225
5,0.00000,0.857576,0.614679,0.909740,0.714437,0.728850,0.0,0.625,0.511548,0.00,0.010204,0.0250,0.000000,0.176711,0.017438,0.5,0.0,0.545455,0.912241,0.5,0.988574,0.1875,0.083333,0.000000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.25,0.25,0.888889,1.000000,0.0,-0.296414,-0.338225
6,0.00000,0.103030,0.214185,0.902617,0.347890,1.000000,0.0,0.750,0.287457,0.15,0.102041,0.0875,0.274194,0.475485,0.052315,0.5,0.0,0.545455,0.990497,0.5,0.997019,0.2500,0.166667,0.166667,0.636364,1.0,0.000000,0.333333,0.571429,0.8,0.25,0.25,0.333333,0.000000,0.0,-0.247027,0.300360
7,0.00000,0.742424,0.102682,0.982205,0.058005,0.432697,0.0,0.875,0.162501,0.60,0.081633,0.0500,0.935484,0.535240,0.000087,0.5,0.0,0.818182,0.054779,0.5,0.984600,0.8125,0.833333,0.833333,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.00,0.00,0.333333,0.000000,0.0,-0.285863,-0.216075
8,0.00015,0.736364,0.226182,0.973662,0.064075,0.503275,0.0,0.875,0.023807,0.60,0.908163,0.4125,0.709677,0.621553,0.003488,0.5,0.0,0.545455,0.428172,0.0,0.992052,0.1875,0.083333,0.000000,0.636364,1.0,0.666667,0.833333,0.857143,0.4,0.50,0.50,0.888889,1.000000,0.0,0.348583,-0.338225
9,0.00000,0.900000,0.402258,0.919928,0.718818,0.279638,0.0,0.625,0.096411,0.30,0.020408,0.0875,0.854839,0.907559,0.000087,0.5,0.0,0.818182,0.011738,0.5,0.995032,0.1875,0.083333,0.000000,0.818182,1.0,0.500000,0.166667,0.857143,0.4,0.25,0.25,0.888889,1.000000,0.0,-0.009892,-0.338225


In [40]:
# PCA for quantity_group and quantity
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(train_scaled[['quantity_group', 'quantity']])
train3['quantity pc'] = pd.DataFrame(data = principalComponents)

#Removing quantity and quantity_group columns
del train3['quantity']
del train3['quantity_group']

train3.head(10)


pca = PCA(n_components=1)
principalComponents = pca.fit_transform(test_scaled[['quantity_group', 'quantity']])
test3['quantity pc'] = pd.DataFrame(data = principalComponents)

#Removing quantity and quantity_group columns
del test3['quantity']
del test3['quantity_group']

test3.head(10)

,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,source,source_type,source_class,funder and installer pc,waterpoint_type pc,quantity pc
0,0.00000,0.772727,0.724418,0.875159,0.648955,0.060522,0.0,0.000,0.463816,0.40,0.204082,0.0375,0.508065,0.008682,0.027988,0.5,0.0,0.181818,0.990497,0.5,0.999503,0.5625,0.500000,0.500000,0.272727,0.5,0.000000,0.333333,0.857143,0.4,0.555556,0.500000,0.5,-0.358319,0.816794,0.591898
1,0.00000,0.772727,0.573747,0.909031,0.713849,0.164222,0.0,0.625,0.321805,0.00,0.010204,0.0250,0.000000,0.331971,0.026157,0.5,0.0,0.545455,0.899944,0.5,0.993542,0.1875,0.083333,0.000000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.888889,1.000000,0.0,-0.296414,-0.338225,0.238345
2,0.00000,0.763636,0.573042,0.862191,0.567270,0.892426,0.0,0.000,0.639346,0.90,0.122449,0.0250,0.879032,0.856486,0.043596,0.5,0.0,0.545455,0.812186,1.0,0.998510,0.5625,0.500000,0.500000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.555556,0.500000,0.5,0.777642,0.816794,0.238345
3,0.00000,0.733333,0.114326,0.943783,0.185559,0.516192,0.0,0.875,0.331043,0.35,0.806122,0.5375,0.395161,0.608784,0.021798,1.0,0.0,0.545455,0.990497,0.5,0.987084,0.5625,0.500000,0.500000,0.636364,1.0,1.000000,1.000000,0.857143,0.4,0.777778,0.833333,0.0,-0.293258,0.816794,-0.468762
4,0.00250,0.927273,0.464714,0.868099,0.053109,0.526432,0.0,0.875,0.419756,0.80,0.091837,0.0375,0.491935,0.549030,0.005231,1.0,0.0,0.818182,0.042482,0.5,0.993542,0.1875,0.083333,0.000000,0.818182,1.0,0.500000,0.166667,0.857143,0.4,0.888889,1.000000,0.0,-0.520495,-0.338225,-0.115208
5,0.00000,0.857576,0.614679,0.909740,0.714437,0.728850,0.0,0.625,0.511548,0.00,0.010204,0.0250,0.000000,0.176711,0.017438,0.5,0.0,0.545455,0.912241,0.5,0.988574,0.1875,0.083333,0.000000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.888889,1.000000,0.0,-0.296414,-0.338225,-0.115208
6,0.00000,0.103030,0.214185,0.902617,0.347890,1.000000,0.0,0.750,0.287457,0.15,0.102041,0.0875,0.274194,0.475485,0.052315,0.5,0.0,0.545455,0.990497,0.5,0.997019,0.2500,0.166667,0.166667,0.636364,1.0,0.000000,0.333333,0.571429,0.8,0.333333,0.000000,0.0,-0.247027,0.300360,-0.115208
7,0.00000,0.742424,0.102682,0.982205,0.058005,0.432697,0.0,0.875,0.162501,0.60,0.081633,0.0500,0.935484,0.535240,0.000087,0.5,0.0,0.818182,0.054779,0.5,0.984600,0.8125,0.833333,0.833333,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.333333,0.000000,0.0,-0.285863,-0.216075,-0.468762
8,0.00015,0.736364,0.226182,0.973662,0.064075,0.503275,0.0,0.875,0.023807,0.60,0.908163,0.4125,0.709677,0.621553,0.003488,0.5,0.0,0.545455,0.428172,0.0,0.992052,0.1875,0.083333,0.000000,0.636364,1.0,0.666667,0.833333,0.857143,0.4,0.888889,1.000000,0.0,0.348583,-0.338225,0.238345
9,0.00000,0.900000,0.402258,0.919928,0.718818,0.279638,0.0,0.625,0.096411,0.30,0.020408,0.0875,0.854839,0.907559,0.000087,0.5,0.0,0.818182,0.011738,0.5,0.995032,0.1875,0.083333,0.000000,0.818182,1.0,0.500000,0.166667,0.857143,0.4,0.888889,1.000000,0.0,-0.009892,-0.338225,-0.115208


In [41]:
# PCA for extraction
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(train_scaled[['extraction_type', 'extraction_type_group']])
train3['extraction_type pc'] = pd.DataFrame(data = principalComponents)

#Removing quantity and quantity_group columns
del train3['extraction_type']
del train3['extraction_type_group']

train3.head(10)

pca = PCA(n_components=1)
principalComponents = pca.fit_transform(test_scaled[['extraction_type', 'extraction_type_group']])
test3['extraction_type pc'] = pd.DataFrame(data = principalComponents)

#Removing quantity and quantity_group columns
del test3['extraction_type']
del test3['extraction_type_group']

test3.head(10)

,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,source,source_type,source_class,funder and installer pc,waterpoint_type pc,quantity pc,extraction_type pc
0,0.00000,0.772727,0.724418,0.875159,0.648955,0.060522,0.0,0.000,0.463816,0.40,0.204082,0.0375,0.508065,0.008682,0.027988,0.5,0.0,0.181818,0.990497,0.5,0.999503,0.500000,0.272727,0.5,0.000000,0.333333,0.857143,0.4,0.555556,0.500000,0.5,-0.358319,0.816794,0.591898,0.237393
1,0.00000,0.772727,0.573747,0.909031,0.713849,0.164222,0.0,0.625,0.321805,0.00,0.010204,0.0250,0.000000,0.331971,0.026157,0.5,0.0,0.545455,0.899944,0.5,0.993542,0.000000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.888889,1.000000,0.0,-0.296414,-0.338225,0.238345,-0.322644
2,0.00000,0.763636,0.573042,0.862191,0.567270,0.892426,0.0,0.000,0.639346,0.90,0.122449,0.0250,0.879032,0.856486,0.043596,0.5,0.0,0.545455,0.812186,1.0,0.998510,0.500000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.555556,0.500000,0.5,0.777642,0.816794,0.238345,0.237393
3,0.00000,0.733333,0.114326,0.943783,0.185559,0.516192,0.0,0.875,0.331043,0.35,0.806122,0.5375,0.395161,0.608784,0.021798,1.0,0.0,0.545455,0.990497,0.5,0.987084,0.500000,0.636364,1.0,1.000000,1.000000,0.857143,0.4,0.777778,0.833333,0.0,-0.293258,0.816794,-0.468762,0.237393
4,0.00250,0.927273,0.464714,0.868099,0.053109,0.526432,0.0,0.875,0.419756,0.80,0.091837,0.0375,0.491935,0.549030,0.005231,1.0,0.0,0.818182,0.042482,0.5,0.993542,0.000000,0.818182,1.0,0.500000,0.166667,0.857143,0.4,0.888889,1.000000,0.0,-0.520495,-0.338225,-0.115208,-0.322644
5,0.00000,0.857576,0.614679,0.909740,0.714437,0.728850,0.0,0.625,0.511548,0.00,0.010204,0.0250,0.000000,0.176711,0.017438,0.5,0.0,0.545455,0.912241,0.5,0.988574,0.000000,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.888889,1.000000,0.0,-0.296414,-0.338225,-0.115208,-0.322644
6,0.00000,0.103030,0.214185,0.902617,0.347890,1.000000,0.0,0.750,0.287457,0.15,0.102041,0.0875,0.274194,0.475485,0.052315,0.5,0.0,0.545455,0.990497,0.5,0.997019,0.166667,0.636364,1.0,0.000000,0.333333,0.571429,0.8,0.333333,0.000000,0.0,-0.247027,0.300360,-0.115208,-0.218587
7,0.00000,0.742424,0.102682,0.982205,0.058005,0.432697,0.0,0.875,0.162501,0.60,0.081633,0.0500,0.935484,0.535240,0.000087,0.5,0.0,0.818182,0.054779,0.5,0.984600,0.833333,0.636364,1.0,0.000000,0.333333,0.857143,0.4,0.333333,0.000000,0.0,-0.285863,-0.216075,-0.468762,0.653623
8,0.00015,0.736364,0.226182,0.973662,0.064075,0.503275,0.0,0.875,0.023807,0.60,0.908163,0.4125,0.709677,0.621553,0.003488,0.5,0.0,0.545455,0.428172,0.0,0.992052,0.000000,0.636364,1.0,0.666667,0.833333,0.857143,0.4,0.888889,1.000000,0.0,0.348583,-0.338225,0.238345,-0.322644
9,0.00000,0.900000,0.402258,0.919928,0.718818,0.279638,0.0,0.625,0.096411,0.30,0.020408,0.0875,0.854839,0.907559,0.000087,0.5,0.0,0.818182,0.011738,0.5,0.995032,0.000000,0.818182,1.0,0.500000,0.166667,0.857143,0.4,0.888889,1.000000,0.0,-0.009892,-0.338225,-0.115208,-0.322644


In [42]:
# PCA for water quality
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(train_scaled[['water_quality', 'quality_group']])
train3['water_quality pc'] = pd.DataFrame(data = principalComponents)

#Removing water quality and quality group columns
del train3['water_quality']
del train3['quality_group']

train3.head(10)

pca = PCA(n_components=1)
principalComponents = pca.fit_transform(test_scaled[['water_quality', 'quality_group']])
test3['water_quality pc'] = pd.DataFrame(data = principalComponents)

#Removing water quality and quality group columns
del test3['water_quality']
del test3['quality_group']

test3.head(10)

,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type_class,management,management_group,payment,payment_type,source,source_type,source_class,funder and installer pc,waterpoint_type pc,quantity pc,extraction_type pc,water_quality pc
0,0.00000,0.772727,0.724418,0.875159,0.648955,0.060522,0.0,0.000,0.463816,0.40,0.204082,0.0375,0.508065,0.008682,0.027988,0.5,0.0,0.181818,0.990497,0.5,0.999503,0.500000,0.272727,0.5,0.000000,0.333333,0.555556,0.500000,0.5,-0.358319,0.816794,0.591898,0.237393,-0.061033
1,0.00000,0.772727,0.573747,0.909031,0.713849,0.164222,0.0,0.625,0.321805,0.00,0.010204,0.0250,0.000000,0.331971,0.026157,0.5,0.0,0.545455,0.899944,0.5,0.993542,0.000000,0.636364,1.0,0.000000,0.333333,0.888889,1.000000,0.0,-0.296414,-0.338225,0.238345,-0.322644,-0.061033
2,0.00000,0.763636,0.573042,0.862191,0.567270,0.892426,0.0,0.000,0.639346,0.90,0.122449,0.0250,0.879032,0.856486,0.043596,0.5,0.0,0.545455,0.812186,1.0,0.998510,0.500000,0.636364,1.0,0.000000,0.333333,0.555556,0.500000,0.5,0.777642,0.816794,0.238345,0.237393,-0.061033
3,0.00000,0.733333,0.114326,0.943783,0.185559,0.516192,0.0,0.875,0.331043,0.35,0.806122,0.5375,0.395161,0.608784,0.021798,1.0,0.0,0.545455,0.990497,0.5,0.987084,0.500000,0.636364,1.0,1.000000,1.000000,0.777778,0.833333,0.0,-0.293258,0.816794,-0.468762,0.237393,-0.061033
4,0.00250,0.927273,0.464714,0.868099,0.053109,0.526432,0.0,0.875,0.419756,0.80,0.091837,0.0375,0.491935,0.549030,0.005231,1.0,0.0,0.818182,0.042482,0.5,0.993542,0.000000,0.818182,1.0,0.500000,0.166667,0.888889,1.000000,0.0,-0.520495,-0.338225,-0.115208,-0.322644,-0.061033
5,0.00000,0.857576,0.614679,0.909740,0.714437,0.728850,0.0,0.625,0.511548,0.00,0.010204,0.0250,0.000000,0.176711,0.017438,0.5,0.0,0.545455,0.912241,0.5,0.988574,0.000000,0.636364,1.0,0.000000,0.333333,0.888889,1.000000,0.0,-0.296414,-0.338225,-0.115208,-0.322644,-0.061033
6,0.00000,0.103030,0.214185,0.902617,0.347890,1.000000,0.0,0.750,0.287457,0.15,0.102041,0.0875,0.274194,0.475485,0.052315,0.5,0.0,0.545455,0.990497,0.5,0.997019,0.166667,0.636364,1.0,0.000000,0.333333,0.333333,0.000000,0.0,-0.247027,0.300360,-0.115208,-0.218587,0.406757
7,0.00000,0.742424,0.102682,0.982205,0.058005,0.432697,0.0,0.875,0.162501,0.60,0.081633,0.0500,0.935484,0.535240,0.000087,0.5,0.0,0.818182,0.054779,0.5,0.984600,0.833333,0.636364,1.0,0.000000,0.333333,0.333333,0.000000,0.0,-0.285863,-0.216075,-0.468762,0.653623,-0.061033
8,0.00015,0.736364,0.226182,0.973662,0.064075,0.503275,0.0,0.875,0.023807,0.60,0.908163,0.4125,0.709677,0.621553,0.003488,0.5,0.0,0.545455,0.428172,0.0,0.992052,0.000000,0.636364,1.0,0.666667,0.833333,0.888889,1.000000,0.0,0.348583,-0.338225,0.238345,-0.322644,-0.061033
9,0.00000,0.900000,0.402258,0.919928,0.718818,0.279638,0.0,0.625,0.096411,0.30,0.020408,0.0875,0.854839,0.907559,0.000087,0.5,0.0,0.818182,0.011738,0.5,0.995032,0.000000,0.818182,1.0,0.500000,0.166667,0.888889,1.000000,0.0,-0.009892,-0.338225,-0.115208,-0.322644,-0.061033


In [43]:
# PCA for payment_type
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(train_scaled[['payment_type', 'payment']])
train3['payment_type pc'] = pd.DataFrame(data = principalComponents)

#Removing payment columns
del train3['payment_type']
del train3['payment']

train3.head(10)


pca = PCA(n_components=1)
principalComponents = pca.fit_transform(test_scaled[['payment_type', 'payment']])
test3['payment_type pc'] = pd.DataFrame(data = principalComponents)

#Removing payment columns
del test3['payment_type']
del test3['payment']

test3.head(10)

,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type_class,management,management_group,source,source_type,source_class,funder and installer pc,waterpoint_type pc,quantity pc,extraction_type pc,water_quality pc,payment_type pc
0,0.00000,0.772727,0.724418,0.875159,0.648955,0.060522,0.0,0.000,0.463816,0.40,0.204082,0.0375,0.508065,0.008682,0.027988,0.5,0.0,0.181818,0.990497,0.5,0.999503,0.500000,0.272727,0.5,0.555556,0.500000,0.5,-0.358319,0.816794,0.591898,0.237393,-0.061033,-0.390102
1,0.00000,0.772727,0.573747,0.909031,0.713849,0.164222,0.0,0.625,0.321805,0.00,0.010204,0.0250,0.000000,0.331971,0.026157,0.5,0.0,0.545455,0.899944,0.5,0.993542,0.000000,0.636364,1.0,0.888889,1.000000,0.0,-0.296414,-0.338225,0.238345,-0.322644,-0.061033,-0.390102
2,0.00000,0.763636,0.573042,0.862191,0.567270,0.892426,0.0,0.000,0.639346,0.90,0.122449,0.0250,0.879032,0.856486,0.043596,0.5,0.0,0.545455,0.812186,1.0,0.998510,0.500000,0.636364,1.0,0.555556,0.500000,0.5,0.777642,0.816794,0.238345,0.237393,-0.061033,-0.390102
3,0.00000,0.733333,0.114326,0.943783,0.185559,0.516192,0.0,0.875,0.331043,0.35,0.806122,0.5375,0.395161,0.608784,0.021798,1.0,0.0,0.545455,0.990497,0.5,0.987084,0.500000,0.636364,1.0,0.777778,0.833333,0.0,-0.293258,0.816794,-0.468762,0.237393,-0.061033,0.810349
4,0.00250,0.927273,0.464714,0.868099,0.053109,0.526432,0.0,0.875,0.419756,0.80,0.091837,0.0375,0.491935,0.549030,0.005231,1.0,0.0,0.818182,0.042482,0.5,0.993542,0.000000,0.818182,1.0,0.888889,1.000000,0.0,-0.520495,-0.338225,-0.115208,-0.322644,-0.061033,-0.086970
5,0.00000,0.857576,0.614679,0.909740,0.714437,0.728850,0.0,0.625,0.511548,0.00,0.010204,0.0250,0.000000,0.176711,0.017438,0.5,0.0,0.545455,0.912241,0.5,0.988574,0.000000,0.636364,1.0,0.888889,1.000000,0.0,-0.296414,-0.338225,-0.115208,-0.322644,-0.061033,-0.390102
6,0.00000,0.103030,0.214185,0.902617,0.347890,1.000000,0.0,0.750,0.287457,0.15,0.102041,0.0875,0.274194,0.475485,0.052315,0.5,0.0,0.545455,0.990497,0.5,0.997019,0.166667,0.636364,1.0,0.333333,0.000000,0.0,-0.247027,0.300360,-0.115208,-0.218587,0.406757,-0.390102
7,0.00000,0.742424,0.102682,0.982205,0.058005,0.432697,0.0,0.875,0.162501,0.60,0.081633,0.0500,0.935484,0.535240,0.000087,0.5,0.0,0.818182,0.054779,0.5,0.984600,0.833333,0.636364,1.0,0.333333,0.000000,0.0,-0.285863,-0.216075,-0.468762,0.653623,-0.061033,-0.390102
8,0.00015,0.736364,0.226182,0.973662,0.064075,0.503275,0.0,0.875,0.023807,0.60,0.908163,0.4125,0.709677,0.621553,0.003488,0.5,0.0,0.545455,0.428172,0.0,0.992052,0.000000,0.636364,1.0,0.888889,1.000000,0.0,0.348583,-0.338225,0.238345,-0.322644,-0.061033,0.443209
9,0.00000,0.900000,0.402258,0.919928,0.718818,0.279638,0.0,0.625,0.096411,0.30,0.020408,0.0875,0.854839,0.907559,0.000087,0.5,0.0,0.818182,0.011738,0.5,0.995032,0.000000,0.818182,1.0,0.888889,1.000000,0.0,-0.009892,-0.338225,-0.115208,-0.322644,-0.061033,-0.086970


In [ ]:
train3.head(10)

# Splitting and Running the Model

In [51]:
# create training and testing vars
y = label.status_group # define the target variable (dependent variable) as y
X_train, X_validation, y_train, y_validation = train_test_split(train3, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_validation.shape, y_validation.shape)

modelRFC = RandomForestClassifier(n_estimators=500,min_samples_split=10,criterion='gini')

(47520, 33) (47520,)
(11880, 33) (11880,)


In [52]:
modelRFC.fit(X_train,y_train)
accuracy_score(modelRFC.predict(X_validation), y_validation)

0.8133838383838384

In [ ]:
accuracy_score(bst.predict(X_validation), y_validation)

In [53]:
# 10-Fold Cross validation
print(np.mean(cross_val_score(modelRFC, train3, y, cv=10)))

0.8190234533986251


In [54]:
predictions = modelRFC.predict(test3)
submission = pd.DataFrame(predictions, columns=['status_group'])
submission.head()

,status_group
0,non functional
1,functional
2,functional
3,non functional
4,functional


In [55]:
test=pd.read_csv("test_set.csv")
#test_scaled = pd.DataFrame(scaler.fit_transform(test), columns=test.columns)
submission.insert(0, 'id', test['id'])
submission.reset_index()
submission.to_csv('submission_watermbd5.csv', index = False)